 ## Import packages and modules

In [2]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col
import eeDatabase_collectionInfo as eedb_colinfo

# ee.Authenticate()
ee.Initialize(project = "climate-engine-pro")

Map = geemap.Map()

## Define Parameters and Load datasets

In [ ]:
# ------------------------------------- Define parameters -----------------------------------------------
# Define time period to export
start_date = datetime.datetime(1980, 1, 1)
end_date = datetime.datetime(2024, 1, 1)

# -------------------------------- Define input Image Collection ----------------------------------------
# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'MOD16_ET'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'PET'

# ------------------------------- Define input Feature Collection ---------------------------------------
# Define input path for Feature Collection
in_fc_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'
in_fc = ee.FeatureCollection(in_fc_path)

# ------------------------------ Define mask, if applicable --------------------------------------------
# For BLM, we will apply mask to field offices, district offices, and state offices, but not to allotments
# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask = True
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'

## Define additional parameters required for functions and output properties below

In [ ]:
# Define land unit names
if(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'):
    land_unit_long = 'BLM_Natl_Grazing_Allotment_Polygons'
    land_unit_short = 'BLM_Allotments'
    in_fc_id = 'ALLOT_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-fieldoffice-polygons'):
    land_unit_long = 'BLM_Natl_FieldOffice_Polygons'
    land_unit_short = 'BLM_FieldOffices'
    in_fc_id = 'FO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'):
    land_unit_long = 'BLM_Natl_DistrictOffice_Polygons'
    land_unit_short = 'BLM_DistrictOffices'
    in_fc_id = 'DO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-stateoffice-polygons'):
    land_unit_long = 'BLM_Natl_StateOffice_Polygons'
    land_unit_short = 'BLM_StateOffices'
    in_fc_id = 'SO_ID'

# Pull out additional variables needed to run exports
in_ic_paths = eedb_colinfo.in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = eedb_colinfo.in_ic_dict.get(in_ic_name).get('var_type')
var_units = eedb_colinfo.var_dict.get(var_name).get('units')
out_path = f"projects/climate-engine-pro/assets/blm-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

## Create database image collection and append time-series images

In [ ]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    eedb_cor.initialize_collection(out_path = out_path, properties = properties)

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:

    print(f"Appending to Image Collection for dates {start_date} - {end_date}")

    # Get dates for image collection based on start and end date
    dates = eedb_cor.get_collection_dates(in_ic_paths = in_ic_paths, start_date = start_date, end_date = end_date)

    # Loop over selected dates to pre-process and export
    for date in dates:
        
        print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
        
        # Parse date for ID
        date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')
        
        # Create dictionary of properties for image    
        properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units}
        if mask == True:
            properties['mask_path'] = mask_path
        elif mask == False:
            properties['mask_path'] = 'None'
        
        eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

{
  "id": "projects/climate-engine-pro/assets/blm-database/blmdistrictoffices-mod16et-et",
  "name": "projects/climate-engine-pro/assets/blm-database/blmdistrictoffices-mod16et-et",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-18T16:08:52.646518Z"
}
{
  "id": "projects/climate-engine-pro/assets/blm-database/blmdistrictoffices-mod16et-et",
  "name": "projects/climate-engine-pro/assets/blm-database/blmdistrictoffices-mod16et-et",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-18T16:08:52.646518Z"
}
Appending to Image Collection for dates 1980-01-01 00:00:00 - 2024-01-01 00:00:00
Running  2000-12-31 17:00:00
Running  2001-01-08 17:00:00
Running  2001-01-16 17:00:00
Running  2001-01-24 17:00:00
Running  2001-02-01 17:00:00
Running  2001-02-09 17:00:00
Running  2001-02-17 17:00:00
Running  2001-02-25 17:00:00
Running  2001-03-05 17:00:00
Running  2001-03-13 17:00:00
Running  2001-03-21 17:00:00
Running  2001-03-29 17:00:00
Running  2001-04-06 18:00:00
Running  2001-04-14 1

## Check completeness of image collections and re-run any images that are missing

In [8]:
# Get list of all dates
all_dates = eedb_cor.get_collection_dates(in_ic_paths, start_date, end_date)

# Get list of dates from collection
coll_dates = ee.ImageCollection(out_path).aggregate_array('system:time_start').distinct().getInfo()

# Get list of dates missing from collection
miss_dates = set(all_dates) - set(coll_dates)
print("These dates are missing and will be rerun ", miss_dates)

for date in miss_dates:
    print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
    
    # Parse date for ID
    date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')

    # Create dictionary of properties for image    
    properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                'var_name': var_name, 'var_units': var_units, 'mask': mask}
    if mask == True:
        properties['mask_path'] = mask_path
    elif mask == False:
        properties['mask_path'] = 'None'
        
    eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

These dates are missing and will be rerun  {1681106400000, 1662098400000, 1680242400000, 1671170400000, 1658642400000, 1667714400000, 1676786400000, 1685858400000, 1664258400000, 1673330400000, 1679810400000, 1662530400000, 1576476000000, 1677218400000, 1661234400000, 1675922400000, 1659938400000, 1682834400000, 1684562400000, 1677650400000, 593848800000, 1674194400000, 1670738400000, 628063200000, 1667282400000, 1663826400000, 1660370400000, 1665986400000, 1674626400000, 1664690400000, 642319200000, 1669442400000, 1679378400000, 1663394400000, 1678082400000, 1675490400000, 1682402400000, 1678946400000, 1672034400000, 1668578400000, 1665122400000, 1661666400000, 1658210400000, 1684130400000, 1680674400000, 887868000000, 1590732000000, 1666850400000, 1681538400000, 1665554400000, 1672898400000, 1669874400000, 1666418400000, 1662962400000, 1659506400000, 1685426400000, 1681970400000, 1678514400000, 1675058400000, 1671602400000, 1668146400000, 1686290400000, 1670306400000, 1684994400000, 

In [49]:
tasks = !geeadd tasks
task_count = int(tasks[1].replace("Tasks Pending: ", ""))
task_count

2325